In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import copy

# Import self-implemented torch dataset
from datasets import BDLibDataset

## Now training and testing on BDLib

In [2]:
window_seconds = 1.0 # The duration per sample in seconds
                     # This affects the sample size (length of the time axis in mel spectrogram),
                     # and also the final accuracy
overlap = 0.75 # The overlap between two samples
n_mels = 32

train_data = BDLibDataset(root='BDLib', 
                          fold_ids=[1,2],
                          download=True,
                          win_secs=window_seconds,
                          overlap=overlap,
                          n_mels=n_mels)
valid_data = BDLibDataset(root='BDLib', 
                          fold_ids=[3],
                          download=False,
                          win_secs=window_seconds,
                          overlap=overlap,
                          n_mels=n_mels)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=16, shuffle=True)

print('train data length', len(train_data))
print('valid data length', len(valid_data))
print('Sample shape', train_data.data[0].shape)

Download not needed, files already on disk.
train data length 2040
valid data length 1020
Sample shape (1, 32, 87)


In [3]:
class timeseries_Encoder():
    def __init__(self,
                 feat_num,
                 quantization_num,
                 D,
                 P,
                 min=0.0,
                 max=1.0):
        self.feat_num = feat_num
        self.quantization_num = int(quantization_num)
        self.D = int(D)
        self.P = float(P)
        self.min = float(min)
        self.max = float(max)
        self.range = max - min
        self.init_hvs()

    def init_hvs(self):
        # level hvs
        num_flip = int(self.D * self.P)
        self.level_hvs = [np.random.randint(2, size=self.D)]
        for i in range(self.quantization_num-1):
            new = copy.deepcopy(self.level_hvs[-1])
            idx = np.random.choice(self.D,num_flip,replace=False)
            new[idx] = 1-new[idx]
            self.level_hvs.append(new)
        self.level_hvs = np.stack(self.level_hvs)

        #id hvs
        self.id_hvs = []
        for i in range(self.feat_num):
            self.id_hvs.append(np.random.randint(2, size=self.D))
        self.id_hvs = np.stack(self.id_hvs)

    def quantize(self, one_sample):
        quantization = self.level_hvs[((((one_sample - self.min) / self.range) * self.quantization_num) - 1).astype('i')]
        return quantization

    def bind(self,a,b):
        return np.logical_xor(a,b).astype('i')

    def permute(self,a):
        for i in range(len(a)):
            a[i] = np.roll(a[i],i,axis=1)
        return a

    def sequential_bind(self,a):
        return np.sum(a,axis=0) % 2

    def bipolarize(self,a):
        a[a==0] = -1
        return a

    def encode_one_time_series_sample(self, one_sample):
        one_sample = one_sample.cpu().numpy()
        T = len(one_sample)
        out = self.quantize(one_sample)
        out = self.bind(out,np.repeat(np.expand_dims(self.id_hvs,0),T,0))
        out = self.permute(out)
        out = self.sequential_bind(out)
        out = self.bipolarize(out)
        out = np.sum(out,axis=0)
        return torch.from_numpy(out).float()

In [4]:
import torchhd
from torchhd.models import Centroid
import torchmetrics

class Encoder(nn.Module):
    def __init__(self, input_dim, levels, hd_dim, flipping, device):
        """
        :param input_dim: dimension of the number of sensors
        :param levels: number of quantized levels
        :param hd_dim: HDC dimension
        :param flipping: flipping ratio to generate the next level hypervector
        :param device: cuda or cpu within torch
        """
        super(Encoder, self).__init__()
        self.device = device
        self.timeseries_Encoder = timeseries_Encoder(input_dim, levels, hd_dim, flipping)

    def forward(self, x):
        x = x.squeeze()
        enc = []
        batch_size = x.shape[0]
        for i in range(batch_size):
            enc.append(
                self.timeseries_Encoder.encode_one_time_series_sample(x[i]).to(self.device))
        sample_hv = torch.stack(enc, dim=0)
        return torchhd.hard_quantize(sample_hv)

In [5]:
def hd_train(train_loader, model, encode, epoch, device):
    with torch.no_grad():
        for samples, labels in tqdm(train_loader, desc="Training"):
            samples = samples.to(device)
            labels = labels.to(device)
    
            # After the following steps, the input data shape should be (batch_size, num_of_time_steps, num_of_sensors)
            samples = samples.squeeze()  # First get rid of the dimension with length=1
            samples = samples.swapaxes(1,2)  # Temp fix for BDLib, need to switch the time and frequency axis
            # print('input sample shape', samples.shape)
            
            samples_hv = encode(samples)
            model.add(samples_hv, labels)
            if epoch > 0: # Retraining
                predict = torch.argmax(model(samples_hv, dot=True), dim=-1)
                wrong_pred = (predict != labels)
                model.add(-samples_hv[wrong_pred], predict[wrong_pred])
                model.add(samples_hv[wrong_pred], labels[wrong_pred])

                    

def hd_test(valid_loader, model, encode, device):
    accuracy = torchmetrics.Accuracy("multiclass", num_classes=train_data.num_classes)
    
    with torch.no_grad():
        model.normalize()
    
        for samples, labels in tqdm(valid_loader, desc="Testing"):
            samples = samples.to(device)
    
            # After the following steps, the input data shape should be (batch_size, num_of_time_steps, num_of_sensors)
            samples = samples.squeeze()  # First get rid of the dimension with length=1
            samples = samples.swapaxes(1,2)  # Temp fix for BDLib, need to switch the time and frequency axis
            
            samples_hv = encode(samples)
            outputs = model(samples_hv, dot=True)
            accuracy.update(outputs.cpu(), labels)
    
    print(f"Testing accuracy of {(accuracy.compute().item() * 100):.3f}%")

In [6]:
HD_DIM = 1000
LEVELS = 10
FLIPPING = 0.005
EPOCHS = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
encode = Encoder(n_mels, LEVELS, HD_DIM, FLIPPING, device)
encode = encode.to(device)

model = Centroid(HD_DIM, train_data.num_classes)
model = model.to(device)

for epoch in range(EPOCHS):
    hd_train(train_loader, model, encode, epoch, device)
    hd_test(valid_loader, copy.deepcopy(model), encode, device)

Testing: 100%|██████████| 64/64 [00:14<00:00,  4.44it/s]


Testing accuracy of 53.137%


Testing: 100%|██████████| 64/64 [00:13<00:00,  4.57it/s]


Testing accuracy of 55.196%


Testing: 100%|██████████| 64/64 [00:13<00:00,  4.64it/s]


...

Testing accuracy of 55.588%


Testing: 100%|██████████| 64/64 [00:14<00:00,  4.55it/s]


Testing accuracy of 55.588%


Testing: 100%|██████████| 64/64 [00:14<00:00,  4.53it/s]

Testing accuracy of 55.686%
